# GtR Topic Classifier

## Preamble

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing useful Python utility libraries
import ast
import re
import smart_open

import numpy as np
import matplotlib.pyplot as plt

from collections import Counter, defaultdict
from itertools import chain, combinations

from im_tutorials.utilities import chunks

In [ ]:
# Pandas is the defacto Python package for transforming and analysing
# tabular data.
# we will refer to pandas as pd to reduce the length of our code
import pandas as pd
pd.set_option('max_columns', 99)

In [ ]:
list_cols = ['research_topics', 'research_subjects']

gtr_projects_df = pd.read_csv(
    smart_open.smart_open('s3://innovation-mapping-tutorials/gateway-to-research/gtr_projects.csv'),
    converters={k: ast.literal_eval for k in list_cols}
)

In [ ]:
gtr_projects_df.head()

In [ ]:
# flatten the lists of research subjects and elements and count the contents
research_subject_counter = Counter(chain(*gtr_projects_df['research_subjects']))
research_topic_counter = Counter(chain(*gtr_projects_df['research_topics']))
print('There are {} unique research subjects in the GtR projects dataset.'.format(len(research_subject_counter)))
print('There are {} unique research topics in the GtR projects dataset.'.format(len(research_topic_counter)))

In [ ]:
print("Top Research Topics by Frequency", '\n')
print('{:<40}{}'.format('Topic', 'Frequency'))
for k, v in research_topic_counter.most_common(20):
    print('{:<40}{}'.format(k, v))
    
print('\nMedian Topic Freqency:')
print(np.median(list(research_topic_counter.values())))

We can see that the top research topic is _Climate & Climate Change_ by some margin. However, we can also see that the top spots are populated by topics from several disciplines. 50% of the topics occur 69 times or fewer, again highlighting the skewness of the distribution.

### Field Definition Through Community Detection

We are going to define communities of research topics as groups of topics which commonly occur together. An effective way of finding these clusters, and visualising the results, is by creating a topic cooccurrence graph.

A cooccurrence graph is a network structure, where nodes are elements and an edge represents the elements of two nodes having cooccured at least once. The edges can then be "weighted" by the frequencies of each cooccurring pair. In the case of our research projects, we can say that two topics have cooccurred if they appear in at least one project together. To find all cooccurrences we therefore need to find the pairwise combinations of research topics for every project. For example, a single project with the topics
```
['Materials Characterisation', 'High Performance Computing', 'Condensed Matter Physics']
```

will become a set of topic pairs:

In [ ]:
# The combinations function from itertools generates all the possible
# elements of combinations from a list with length  r.
list(combinations(['Materials Characterisation', 'High Performance Computing', 'Condensed Matter Physics'], 2))

These cooccurrences would form a triangular network, where each edge has a frequency weight of 1.

To create a cooccurrence network across all projects, we need to repeat this process for every project. We can do this in a Python list comprehension, and then chain togeher all of the cooccurring pairs into one long list.

In [ ]:
# Generate every pairwise combination of research topics from each project.
# Each pair is sorted alphabetically to make sure that there is only one 
# possible permutation of each edge.
cooccurrences = list(chain(*[sorted(itertools.combinations(d, 2)) for d in gtr_projects_df['research_topics']]))
# Count the frequency of each cooccurring pair.
research_topic_edge_counter = Counter(cooccurrences)

In [ ]:
print("Top Research Topic Cooccurrences by Frequency", '\n')
print('{:<70}{}'.format('Cooccurrence', 'Frequency'))
for k, v in research_topic_edge_counter.most_common(20):
    print('{:<70}{}'.format((k[0] + ' + ' +k[1]), v))

Looking at the most frequently cooccurring topics we can pairs that make intuitive sense and are all generally captured neatly within higher order academic disciplines.

However this, along with the individual topic frequencies, also shows us that using the cooccurrence frequency as our edge weight might not be such a good idea. High frequency elements are simply more likely to cooccur due to chance. Therefore we should normalise our edge weights. One method for this is to calculate the association strength, a proababilistic measure, where the cooccurrence freqency is normalised by the product of the individual terms' occurrence counts. It is defined as

$$ a = \frac{2 n c_{ij}}{o_{i}o_{j}} $$

where $n$ is the total number of elements, $c_{ij}$ is the number of cooccurrences between elements $i$ and $j$, and $o_{i}$ and $o_{j}$ are the individual frequency counts of each element.

In [ ]:
def association_strength(combo, occurrences, cooccurrences, total):
    '''association_strength
    Calculates the association strength between a cooccurring pair.
    '''
    a_s = ((2 * total * cooccurrences[combo]) / 
           (occurrences[combo[0]] * occurrences[combo[1]]))
    return a_s

To build our cooccurrence network, we need to generate a list of unique edges from our long list of cooccurrences and then calculate the association strength for each edge.

In [ ]:
# Generate a set of cooccurences (a list of unique pairs).
# This will form the edges of our cooccurrence graph.
edges = set(cooccurrences)
# Calculate the total number of elements
n = len(list(chain(*gtr_projects_df['research_topics'])))
# Calculate the association strength for each edge.
# We take the log of the association strength to give it
# a normal distribution.
assoc_strengths = np.log10([association_strength(
    edge,
    research_topic_counter, 
    research_topic_edge_counter, 
    total_research_topics) for edge in edges])

In [ ]:
fig, ax = plt.subplots()
ax.hist(assoc_strengths, bins=100)
ax.set_xlabel('Association Strength')
plt.show()

### Build the Graph

In [ ]:
import networkx as nx

In [ ]:
weighted_edges = []
for (s, t), a_s in zip(edges, assoc_strengths):
    weighted_edges.append((s, t, a_s))


g = nx.Graph()
g.add_weighted_edges_from(weighted_edges, weight='association_strength')

In [ ]:
print(g.edges[('Materials Characterisation','Materials Synthesis & Growth')])

### Community Detection

In [ ]:
import community

In [ ]:
part = community.best_partition(g, resolution=0.7, random_state=1, weight='weight')
n_communities = len(set(part.values()))
print('{} communities detected.'.format(n_communities))

### Interactive Network Visualisation

First we add some extra properties to the nodes in our graph.

In [ ]:
names = {k: k for k, _ in part.items()}
nx.set_node_attributes(g, names, name='topic_name')
community_colors = {k: Category20[n_communities][c] for k, c in part.items()}
nx.set_node_attributes(g, community_colors, name='color')

print(g.nodes['Materials Characterisation'])

Then we calculate positions for the visual graph layout.

In [ ]:
pos = nx.spring_layout(g, weight='association_strength', scale=2, random_state=0)

`bokeh` has built-in support for `networkx` graphs, which makes plotting and interacting with them easy.

In [ ]:
plot = figure(title="Research Topic Cooccurrence Network",
              x_range=(-2.1,2.1), y_range=(-2.1,2.1),
             )

graph_renderer = from_networkx(g, pos, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(size=7, fill_color='color', line_color=None)
graph_renderer.node_renderer.selection_glyph = Circle(size=7, fill_color='color')
graph_renderer.node_renderer.hover_glyph = Circle(size=7, fill_color='color')
graph_renderer.node_renderer.muted_glyph = Circle(size=7, fill_color='color', fill_alpha=0.9)


graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.2, line_width=1)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=1.5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=1.5)

graph_renderer.selection_policy = NodesAndLinkedEdges()

node_hover_tool = HoverTool(tooltips=[("Topic", "@topic_name")])
plot.add_tools(node_hover_tool, TapTool())

plot.renderers.append(graph_renderer)

show(plot)

### Investigating the Communities

Let's manually inspect the topics in each community to see if we can see what disciplines they might form.

In [ ]:
reverse_part = defaultdict(list)
for k, v in part.items():
    reverse_part[v].append(k)
    
for c, topics in reverse_part.items():
    print(c)
    for chunk in chunks(topics, 4):
        print(', '.join(chunk))
    print('')

We can now create a community ID to discipline mapping.

In [ ]:
discipline_map = {
    0: ,
    1: ,
    2: ,
    3: ,
    4: ,
    5: ,
    6: ,
}

## Extra Stuff

In [ ]:
class CommunityPartition:
    def __init__(self, graph):
        self.graph = graph
    
    def edgelist_to_cooccurrence(self, repeats, **best_partition_kwargs):
        edge_counter = Counter()
        for i in range(repeats):
            partition = community.best_partition(self.graph, random_state=i, **best_partition_kwargs)
            edgelist = self.partition_to_edgelist(partition)
            edge_counter.update(edgelist)

        g = nx.Graph()
        g.add_weighted_edges_from([(e[0][0], e[0][1], e[1]) for e in edge_counter.items()])
        return g
    
    def partition_to_edgelist(self, partition):
        partition_reverse_mapping = self.reverse_index_partition(partition)
        edgelist = []
        for community, elements in partition_reverse_mapping.items():
            combos = [tuple(sorted(e)) for e in itertools.combinations(elements, 2)]
            edgelist.extend(combos)
        return edgelist
     
    def reverse_index_partition(self, partition):
        partition_reverse_mapping = defaultdict(list)
        for k, v in partition.items():
            partition_reverse_mapping[v].append(k)
        return partition_reverse_mapping

In [ ]:
cp = CommunityPartition(g)

In [ ]:
co = cp.edgelist_to_cooccurrence(3, resolution=.8)

In [ ]:
#Extract the best partition
part = community.best_partition(g, resolution=0.8, random_state=0, weight='weight')

In [ ]:
reverse_partition = defaultdict(list)
for k, v in part.items():
    reverse_partition[v].append(k)
    
names = {k: dictionary[k] for k, _ in part.items()}

In [ ]:
names = {k: dictionary[k] for k, _ in part.items()}
nx.set_node_attributes(co, part, name='community')
nx.set_node_attributes(co, names, name='topic')
community_colors = {k: Category20[20][c] for k, c in part.items()}
nx.set_node_attributes(co, community_colors, name='color')

In [ ]:
set(part.values())

In [ ]:
pos = nx.spring_layout(co, weight='weight', scale=2, random_state=0)

In [ ]:
import pickle

In [ ]:
category_name_lookup = {
    0: 'social_sciences',
    1: 'arts_linguistics',
    2: 'chem_mater_phys_eng',
    3: 'social_sciences',
    4: 'maths_computing_ee',
    5: 'social_sciences',
    6: 'social_sciences',
    7: 'biological_sciences',
    8: 'social_sciences',
    9: 'humanities',
    10: 'arts_linguistics',
    11: 'humanities',
    12: 'social_sciences',
    13: 'physics',
    14: 'environmental_sciences',
    15: 'social_sciences',
    16: 'humanities'
}

topic_discipline_lookup = {top:category_name_lookup[disc] for top, disc in part.items()}

In [ ]:
with open(os.path.join(model_path, 'communities_partition.pkl'), 'wb') as f:
    pickle.dump(part, f)

with open(os.path.join(model_path, 'communities_partition_labels.pkl'), 'wb') as f:
    pickle.dump(category_name_lookup, f)

In [ ]:
gtr_projects_df['discipline'] = gtr_projects_df['research_topics'].apply(
    lambda x: [topic_discipline_lookup[val] for val in x])

gtr_projects_df['discipline_sets'] = [set(x) for x in gtr_projects_df['discipline']]

gtr_projects_df['single_disc'] = [True if len(x)==1 else np.nan if len(x)==0 else False for x in gtr_projects_df['discipline_sets']]

gtr_projects_df['single_disc'].mean()

In [ ]:
gtr_projects_df['discipline_sets'] = [
    set(['medical_sciences']) if f =='MRC' else x for f,x in zip(
        gtr_projects_df['funder_name'],
           gtr_projects_df['discipline_sets'])]

In [ ]:
def modal_value(l):
    c = Counter(l)
    try:
        return c.most_common(1)[0][0]
    except:
        return np.nan

gtr_projects_df['modal_discipline'] = [modal_value(d) for d in gtr_projects_df['discipline_sets']]

In [ ]:
gtr_projects_df['modal_discipline'].value_counts()

In [ ]:
Counter(chain(*gtr_projects_df['discipline_sets'])).most_common()

In [ ]:
n_labels = [True if len(s) > 0 else False for s in gtr_projects_df['discipline_sets']]

In [ ]:
# remove projects without abstracts
gtr_projects_df = gtr_projects_df[~pd.isnull(gtr_projects_df['abstract_texts'])]
# remove projects with short abstracts
gtr_projects_df = gtr_projects_df[gtr_projects_df['abstract_texts'].str.len() > 250]
# remove projects with no labels
n_labels = [True if len(s) > 0 else False for s in gtr_projects_df['discipline_sets']]
gtr_projects_df = gtr_projects_df[n_labels]

### Text Preprocessing

In [ ]:
import spacy
from gensim.models.phrases import Phraser

In [ ]:
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')

with open(os.path.join(raw_data_path, 'stopwords_en_long.txt'), 'r') as f:
    stopwords = f.read().splitlines()

for stopword in stopwords:
    nlp.vocab[stopword.lower()].is_stop = True
    nlp.vocab[stopword.upper()].is_stop = True
    nlp.vocab[stopword.title()].is_stop = True

In [ ]:
abstracts = [remove_markup(a) for a in gtr_projects_df['abstract_texts']]
abstracts = [normalise_digits(a) for a in abstracts]
abstracts = lemmatize(abstracts, nlp)

In [ ]:
bigrammer = Phraser.load(os.path.join(model_path, 'gtr_discipline_bigrammer.pkl'))
abstracts = bigram(abstracts, phraser=bigrammer)
abstracts_str = list(stringify_docs(abstracts))

In [ ]:
gtr_projects_df['abstract_processed'] = abstracts_str

### Single Label

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

In [ ]:
gtr_projects_df['n_disciplines'] = [len(a) for a in gtr_projects_df['discipline_sets']]
gtr_pure_df = gtr_projects_df[gtr_projects_df['n_disciplines'] == 1]

In [ ]:
tfidf_single = TfidfVectorizer(
    max_df=0.4, 
    min_df=5,
    sublinear_tf=True, 
    norm='l2'
)
tfidf_pure_vecs = tfidf_single.fit_transform(gtr_pure_df['abstract_processed'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_pure_vecs, gtr_pure_df['modal_discipline'], train_size=0.8, test_size=0.2)

In [ ]:
sp = SelectPercentile(chi2, 40)
sp_vecs = sp.fit_transform(X_train, y_train)
sp_vecs_test = sp.transform(X_test)

In [ ]:
tfidf_vecs.shape

In [ ]:
sp_vecs.shape

In [ ]:
lr = LogisticRegression(C=10, solver='lbfgs', multi_class='auto')
mnb = MultinomialNB()
cmb = ComplementNB()
voter = VotingClassifier(estimators=[('lr', lr), ('cmb', cmb), ('mnb', mnb)])

In [ ]:
pipe = Pipeline??

In [ ]:
from sklearn.model_selection import GridSearchCV

#### Optimise

In [ ]:
lr_params = {'C': [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100]}
lr_grid = GridSearchCV(lr, param_grid=lr_params, cv=5, verbose=2)

In [ ]:
lr_grid.fit(sp_vecs, y_train)
lr_best = lr_grid.best_estimator_
print(classification_report(y_test, lr_best.predict(sp_vecs_test)))

In [ ]:
sns.heatmap(
    pd.DataFrame(confusion_matrix(y_test, lr_best.predict(sp_vecs_test)), columns=mlb.classes_, index=mlb.classes_),
    annot=True,
    fmt='d'
)

#### Export Pipe

In [ ]:
tfidf_single = TfidfVectorizer(
    max_df=0.4, 
    min_df=5,
    sublinear_tf=True, 
    norm='l2'
)
sp = SelectPercentile(chi2, 40)
lr = LogisticRegression(C=10, solver='lbfgs', multi_class='auto')
pipe = Pipeline(
    steps=[
        ('tfidf', tfidf),
        ('sp', sp),
        ('lr', lr)
    ]
)

In [ ]:
pipe.fit(gtr_pure_df['abstract_processed'], gtr_pure_df['modal_discipline'])

In [ ]:
joblib.dump(pipe, os.path.join(model_path, 'gtr_discipline_lvl9_lr_20190222.pkl'))

### Multilabel

In [ ]:
tfidf = TfidfVectorizer(
    max_df=0.3, 
    min_df=5,
    sublinear_tf=True, 
    norm='l2'
)
tfidf_vecs = tfidf.fit_transform(abstracts_str)

In [ ]:
classes = list(set(chain(*gtr_projects_df['discipline_sets'])))
mlb = MultiLabelBinarizer(classes=classes)
target_binarized = mlb.fit_transform(gtr_projects_df['discipline_sets'])
target_binarized_df = pd.DataFrame(target_binarized, columns=mlb.classes_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_vecs, target_binarized_df, train_size=0.8, test_size=0.2)
sp = SelectPercentile(chi2, 40)
sp_vecs = sp.fit_transform(X_train, y_train)
sp_vecs_test = sp.transform(X_test)

In [ ]:
lr = LogisticRegression(solver='lbfgs', multi_class='auto')
mnb = MultinomialNB()
cmb = ComplementNB()
voter = VotingClassifier(estimators=[('lr', lr), ('cmb', cmb), ('mnb', mnb)])

In [ ]:
for cls in mlb.classes_:
    print(cls)
    lr.fit(sp_vecs, y_train[cls])
    print(classification_report(y_test[cls], lr.predict(sp_vecs_test)))

In [ ]:
print(classification_report(y_test, voter.predict(sp_vecs_test)))

### Train RF

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_params = {
     'bootstrap': [True, False],
     'max_depth': [10, 20, 50, 100, None],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [100, 200, 500]
}

In [ ]:
rf = RandomForestClassifier(n_jobs=3)

rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_params,
    n_iter=100,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=3
)

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test, rf_random.best_estimator_.predict(X_test)))

In [ ]:
rf_random.best_estimator_.fit(tfidf_vecs_filt, target_binarized)

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(rf_random.best_estimator_, os.path.join(model_path, 'gtr_discipline_classifier_rf_8_20192102.pkl'))
joblib.dump(tfidf, os.path.join(model_path, 'gtr_discipline_tfidf_20192102.pkl'))
bigrammer.save(os.path.join(model_path, 'gtr_discipline_bigrammer.pkl'))
nlp.vocab.to_disk(os.path.join(model_path, 'gtr_discipline_vocab'))

## Apply to CORDIS

In [ ]:
cordis_projects_df = pd.read_csv(os.path.join(inter_data_path, 'fp7_h2020_projects.csv'))

In [ ]:
cordis_abstracts = [remove_markup(a) for a in cordis_projects_df['objective'][:25]]
cordis_abstracts = [normalise_digits(a) for a in cordis_abstracts]
cordis_abstracts = lemmatize(cordis_abstracts, nlp)
cordis_abstracts = bigram(cordis_abstracts, phraser=bigrammer)
cordis_abstracts = list(stringify_docs(cordis_abstracts))

In [ ]:
for abstract, pred in zip(cordis_projects_df['objective'][:25], pipe.predict(cordis_abstracts)):
    print(pred)
    print(abstract)
    print('\n==============')

In [ ]:
cordis_tfidf_vecs = tfidf.transform(cordis_abstracts)

In [ ]:
cordis_subject_probs = rf_random.best_estimator_.predict_proba(cordis_tfidf_vecs)
cordis_subjects = rf_random.best_estimator_.predict(cordis_tfidf_vecs)

In [ ]:
subject_probs = np.zeros((len(cordis_projects_df), 8))

In [ ]:
for i in range(8):
    subject_probs[:, i] = cordis_subject_probs[i][:, 0]

In [ ]:
n = 101

In [ ]:
cordis_projects_df['objective'][n]

In [ ]:
pd.DataFrame(cordis_subjects, columns=mlb.classes_).sum()

## Alternative Feature Selection

In [ ]:
feature_terms = []
indices = np.array(range(0, X_train.shape[1]))
for discipline in y_train.columns:
    features_chi2 = chi2(X_train, y_train[discipline])[0]
    threshold = np.percentile(features_chi2[~pd.isnull(features_chi2)], 90)
    discipline_indices = indices[features_chi2 > threshold]
    feature_terms.extend(np.array(tfidf.get_feature_names())[discipline_indices])

In [ ]:
tfidf_stop_words = set(tfidf.get_feature_names()).difference(set(feature_terms))

In [ ]:
tfidf = TfidfVectorizer(
#     max_df=0.5, 
    min_df=5, 
    sublinear_tf=True, 
    norm='l2',
    stop_words=tfidf_stop_words
)
tfidf_vecs_filt = tfidf.fit_transform(abstracts_str)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_vecs_filt, target_binarized, train_size=0.9, test_size=0.1)